<a href="https://colab.research.google.com/github/nayun12/Tobigs_17/blob/main/week7_nlp_basic_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP 기초 Assignment
## NLP 맛보기 - spam.csv를 가지고 유의미한 해석을 도출해주세요!

In [129]:
# 필요한 모듈 불러오기
import pandas as pd

In [130]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Data
- 보시면 아시다시피 spam.csv는 라벨이 있는 데이터입니다. 물론 7주차 주제가 텍스트 기초인만큼 텍스트만 활용하셔도 되고, 라벨까지 활용하셔서 모델을 돌려보셔도 좋습니다 :)

In [131]:
spam = pd.read_csv('/content/spam.csv')

In [132]:
spam['v2'] = spam['v2'].str.lower()

In [133]:
spam.head()

,v1,v2
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [134]:
spam.shape

(5572, 2)

In [135]:
spam['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [136]:
spam.isnull().sum()

v1    0
v2    0
dtype: int64

In [137]:
spam['v2'].describe()

count                       5572
unique                      5168
top       sorry, i'll call later
freq                          30
Name: v2, dtype: object

In [138]:
spam = spam.drop_duplicates('v2')
len(spam)

5168

## Tokenizing


In [139]:
import nltk

In [140]:
import re
word = re.sub('[^a-zA-Z]', ' ', spam.iloc[5]['v2'])
words = word_tokenize(word)
print(words)

['freemsg', 'hey', 'there', 'darling', 'it', 's', 'been', 'week', 's', 'now', 'and', 'no', 'word', 'back', 'i', 'd', 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', 'tb', 'ok', 'xxx', 'std', 'chgs', 'to', 'send', 'to', 'rcv']


In [141]:
from nltk.corpus import stopwords

nltk.download('stopwords')
english_stops = set(stopwords.words('english'))
words = [word for word in words if word not in english_stops]
print(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['freemsg', 'hey', 'darling', 'week', 'word', 'back', 'like', 'fun', 'still', 'tb', 'ok', 'xxx', 'std', 'chgs', 'send', 'rcv']


In [142]:
spam.iloc[5]['v2']

"freemsg hey there darling it's been 3 week's now and no word back! i'd like some fun you up for it still? tb ok! xxx std chgs to send, å£1.50 to rcv"

In [143]:
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word) for word in words]
print(words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
['freemsg', 'hey', 'darling', 'week', 'word', 'back', 'like', 'fun', 'still', 'tb', 'ok', 'xxx', 'std', 'chgs', 'send', 'rcv']


In [144]:
spam_token = []
for i in range(len(spam)):
    words = re.sub('[^a-zA-Z]', ' ', spam.iloc[i]['v2'])  # 특수문자 및 숫자 제거
    words = word_tokenize(words)  # tokenize
    words = [word for word in words if word not in english_stops]  # 불용어 제거
    words = [lemmatizer.lemmatize(word) for word in words]  # 표제어추출
    words = [word.lower() for word in words]  # 소문자 변환
    spam_token.append(words)

In [148]:
spam_token

[['go',
  'jurong',
  'point',
  'crazy',
  'available',
  'bugis',
  'n',
  'great',
  'world',
  'la',
  'e',
  'buffet',
  'cine',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'u', 'oni'],
 ['free',
  'entry',
  'wkly',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'c',
  'apply'],
 ['u', 'dun', 'say', 'early', 'hor', 'u', 'c', 'already', 'say'],
 ['nah', 'think', 'go', 'usf', 'life', 'around', 'though'],
 ['freemsg',
  'hey',
  'darling',
  'week',
  'word',
  'back',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'send',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'melle',
  'melle',
  'oru',
  'minnaminunginte',
  'nurungu',
  'vettam',
  'set',
  'callertune',
  'caller',
  'press',
  'copy',
  'friend',
  'callertune'],
 ['winner',
  'valued',
  'network',
  

## Embedding

- 수업에서 다룬 임베딩 방법에는 One-hot encoding, CBOW, Skip-gram, GloVe, FastText, EMlo가 있었습니다. 다양한 시도와 '비교' 결과를 함께 적어주세요! 파라미터를 조정해가는 과정도 해석에 도움이 될 수 있겠죠 :)

In [158]:
from gensim.models import Word2Vec,FastText
import numpy as np

In [149]:
CBOW = Word2Vec(spam_token, size=2, window=2, min_count=1, workers=1, iter=1000, sg=0)
CBOW_words = CBOW.wv.index2word
CBOW_vectors = CBOW.wv.vectors

In [150]:
CBOW_df = pd.DataFrame(CBOW_vectors, columns = ['x1', 'x2'])
CBOW_df['words'] = CBOW_words
CBOW_df = CBOW_df[['words', 'x1', 'x2']]
CBOW_df

,words,x1,x2
0,u,0.219765,-0.818935
1,call,-2.230982,-0.394487
2,get,-0.542159,-0.590397
3,ur,-1.037609,-0.442243
4,go,0.861014,-1.055312
...,...,...,...
7034,heap,-0.587069,-0.598204
7035,lowes,3.360158,-2.465508
7036,salesman,1.654333,-1.245996
7037,pity,-0.534560,-1.196365


In [151]:
CBOW.most_similar('week', topn=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('qlynnbv', 1.0),
 ('onluy', 1.0),
 ('canlove', 0.9999999403953552),
 ('promotion', 0.9999999403953552),
 ('president', 0.9999999403953552)]

In [152]:
CBOW.most_similar('happy', topn=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('child', 0.9999999403953552),
 ('ything', 0.9999998211860657),
 ('shouted', 0.9999998211860657),
 ('triumphed', 0.9999995827674866),
 ('prepared', 0.9999989867210388)]

In [153]:
CBOW.most_similar('customer', topn=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('lived', 1.0),
 ('cc', 1.0),
 ('mnths', 0.9999999403953552),
 ('hb', 0.9999999403953552),
 ('whose', 0.9999998807907104)]

In [154]:
Skip_Gram = Word2Vec(spam_token, size=100, window=5, min_count=1, workers=1, iter=1000, sg=1)
Skip_Gram_words = Skip_Gram.wv.index2word
Skip_Gram_vectors = Skip_Gram.wv.vectors

Skip_Gram_df = pd.DataFrame(Skip_Gram_vectors, columns = ['x1', 'x2'])
Skip_Gram_df['words'] = Skip_Gram_words
Skip_Gram_df = Skip_Gram_df[['words', 'x1', 'x2']]

In [155]:
Skip_Gram.most_similar('week', topn=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('website', 0.9999998807907104),
 ('njan', 0.9999997615814209),
 ('axis', 0.9999996423721313),
 ('dificult', 0.9999993443489075),
 ('forwarded', 0.9999992847442627)]

In [156]:
Skip_Gram.most_similar('happy', topn=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('stubborn', 1.0),
 ('gravity', 0.9999999403953552),
 ('dream', 0.9999998807907104),
 ('look', 0.9999998807907104),
 ('greece', 0.9999998211860657)]

In [157]:
Skip_Gram.most_similar('customer', topn=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('awaiting', 1.0),
 ('handset', 1.0),
 ('xxxxxx', 1.0),
 ('tg', 1.0),
 ('pura', 1.0)]

In [159]:
Fastext = FastText(spam_token, size=100, window=5, min_count=5, workers=4, sg=1)
Fastext_words = Fastext.wv.index2word
Fastext_vectors = Fastext.wv.vectors

In [160]:
Fastext.most_similar('week', topn=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('weekly', 0.9981003999710083),
 ('ringtone', 0.9963485598564148),
 ('poly', 0.9956563711166382),
 ('txtauction', 0.9947611093521118),
 ('txts', 0.9946895241737366)]

In [161]:
Fastext.most_similar('happy', topn=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('love', 0.9942669868469238),
 ('make', 0.9933640956878662),
 ('something', 0.9926269054412842),
 ('anything', 0.9917293787002563),
 ('good', 0.9917188882827759)]

In [162]:
Fastext.most_similar('customer', topn=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('camera', 0.9986586570739746),
 ('service', 0.9981647729873657),
 ('entry', 0.9971777200698853),
 ('mobileupd', 0.9969955086708069),
 ('camcorder', 0.9964990615844727)]

## 본인이 도출해낸 해석을 적어주세요!

- 유사도, Wordcloud, 이진 분류 모델, Plot 뭐든 상관없으니 분명하고 인상적인 해석을 적어주시면 됩니다.

임베딩 모델의 유사도를 비교해본 결과 기본적으로 임베딩 차원 size를 늘렸을 때 좋은 결과가 도출되었다.

Skip_gram보다 FastText가 조금이나마 나은 결과를 보였다.

그러므로 최종 선택은 FastText로 결정하였다.